# AiDM Assignment 2
##  Locality Sensitive Hashing for Netflix Data

In [222]:
import numpy as np
import time
import os
start = time.time()

###  Load Data
First we read in the data

In [223]:
data_origin = np.load('../user_movie.npy')

We show the function for calculating the similarities between users here.

In [224]:
def generate_list(data,n):
    '''
    The function is to generate a list that contains sets for users. Each set is the watched movies for one user.
    data :
    n : persent of dataset will be considered
    '''
    data_n_persent = data[0:int(len(data_origin)*n),:] # Drag out n persent from the entire dataset.
    user_movie_list = [[] for _ in range(len(set(data_n_persent[:,0])))] 
    # Prepare a list of sets.
    # Every set represent a user  
    for i in range(len(data_n_persent)):
        user_movie_list[data_n_persent[i,0]].append(data_n_persent[i,1])
    for j in range(len(user_movie_list)):
        user_movie_list[j] = set(user_movie_list[j]) # convert every user list to a set
    return user_movie_list  # will return the whole watched movies set for every user in only one list.
def simi(pair,set_list):
    a = set_list[pair[0]]
    b = set_list[pair[1]]
    intersection = a & b # Use logical and to achieve the intersection
    n_intersection = len(intersection)
    similarity = n_intersection / (len(a) + len(b) - n_intersection) # Calculate sim
    return similarity

In [225]:
set_list_total = generate_list(data_origin,1)

### Shingling
In the interest of memory conservation (only stoing non-zero elements) and efficient rearrangement, we store the data as ones in indicies according to their user ID (column) and rated movies (rows, 1 if rated 0 if not) 

In [226]:
from scipy.sparse import csc_matrix
users_movies = csc_matrix((np.ones(len(data_origin)), (data_origin[:,1], data_origin[:,0])),\
                          shape=(np.max(data_origin[:,1])+1, np.max(data_origin[:,0])+1))

### Signature Matrix
Now we create a random permutations of movie indicies and rearrange our matrix to match. The first non-zero element in the new arrangement will be an element in the user's signature. We will start using 100 permutations, so that our signatures for each user are of length 100.

In [227]:
tic = time.time()
signature_length = 50
signature_matrix = np.zeros((signature_length, np.max(data_origin[:,0])+1))
permutation_indicies = np.arange(np.max(data_origin[:,1])+1)
users_movies_copy = np.copy(users_movies)
for i in range(signature_length):
    np.random.shuffle(permutation_indicies)
    users_movies_copy = users_movies[permutation_indicies,:]
    signature_matrix[i,:]= (users_movies_copy!=0).argmax(axis=0)
print('Gee-whiz, it only took', int(time.time() - tic), 'seconds to make the signature matrix!')

Gee-whiz, it only took 219 seconds to make the signature matrix!


Now let's test to make sure we made our signture matrix correctly. For the last signature in the matrix for the first user, is the index actually the first non-zero entry in the permutation_indicies arrangement?


In [228]:
for j in permutation_indicies[1+np.arange(int(signature_matrix[signature_length-1,0]))]:
    print('Index:', j, ', Entry:', users_movies[j,0])

Index: 16806 , Entry: 0.0
Index: 897 , Entry: 0.0
Index: 2337 , Entry: 0.0
Index: 16793 , Entry: 0.0
Index: 3342 , Entry: 0.0
Index: 16833 , Entry: 0.0
Index: 5667 , Entry: 0.0
Index: 4686 , Entry: 0.0
Index: 13188 , Entry: 0.0
Index: 382 , Entry: 0.0
Index: 16623 , Entry: 0.0
Index: 7923 , Entry: 0.0
Index: 14838 , Entry: 0.0
Index: 14575 , Entry: 0.0
Index: 4793 , Entry: 0.0
Index: 3691 , Entry: 0.0
Index: 15223 , Entry: 0.0
Index: 6969 , Entry: 0.0
Index: 12780 , Entry: 0.0
Index: 8782 , Entry: 0.0
Index: 14517 , Entry: 0.0
Index: 7232 , Entry: 0.0
Index: 9621 , Entry: 0.0
Index: 4247 , Entry: 0.0
Index: 7844 , Entry: 0.0
Index: 14873 , Entry: 0.0
Index: 1733 , Entry: 0.0
Index: 3452 , Entry: 0.0
Index: 3101 , Entry: 0.0
Index: 2912 , Entry: 1.0


Yep, that is indeed the first non-zero entry in this ordering!

###  Partition Into Bands
Next we cut the signature matrix into 5 bands (to start),

In [236]:
def hash_to_bucket(bands = 5,sig_M = np.ones((100,20))):
    rows = int(len(sig_M) / bands)
    possible_sig_max = 17749
    candidate_list = [[[] for _ in range(possible_sig_max * rows)] for _ in range(bands)]
    candidates = []
    for i in range(bands):
        for j in range(len(sig_M[0])):
            hash_sum = sum(sig_M[int(i * rows):int((i+1)*20),j])
            candidate_list[i][int(hash_sum)].append(j)
        candidates.append([x for x in candidate_list[i] if x])
    return candidates

def bucket_to_simi(candidates, start = time.time()):
    pairs = []
    similarities = []
    for band in candidates:
        for bucket in band:
            for i in range(len(bucket)):
                for j in np.arange(start=i+1, stop = len(bucket), step=1):
                    pair = [bucket[i],bucket[int(j)]]
                    if pair not in pairs:
                        similarity = simi(pair,set_list_total)
                        pairs.append(pair)
                        similarities.append(simi(pair,set_list_total))
                    if time.time() - start > 30:
                        break
    return pairs , similarities

In [239]:
       
candidates = hash_to_bucket(bands = 5, sig_M = signature_matrix)
pairs , similarities = bucket_to_simi(candidates)
Threshold = 0.5
maker = similarities > 0.5
similar_users = pairs[maker]

if os.path.exists('results.txt'):
    os.remove('results.txt')
with open('results.txt','w') as f:
    for item in similar_users:
        f.write('%s,%s\n' % (item[0],item[1]))
f.close()
pairs

KeyboardInterrupt: 

In [240]:
pairs

[[73744, 77806]]

In [231]:
"""
# Meng's testing

#candidates = set().union(candidate_list[0],candidate_list[1])
#candidate_list[0] | candidate_list[1]
#candidates[0] == candidates[1]
'''
candidates_unique = candidates[0]
for i in range(bands-1):
    candidates_unique += [e for e in candidates[i+1] if e not in candidates_unique]
candidates_unique
''''


"""

'\ncandidates_unique = candidates[0]\nfor i in range(bands-1):\n    candidates_unique += [e for e in candidates[i+1] if e not in candidates_unique]\ncandidates_unique\n'

In [232]:
'''
#Michael's testing

row = np.array([0, 2, 2, 0, 1, 2])
col = np.array([0, 0, 1, 2, 2, 2])
data = np.array([1, 2, 3, 4, 5, 6])
A = csc_matrix((data, (row, col)), shape=(len(row), len(col))).toarray()
print(A)
x = np.arange(len(row))
np.random.shuffle(x)
print(x)
B = A[x,:]
print(B)
A[0,:] = (B!=0).argmax(axis=0)
print(A)
'''

[[1 0 4 0 0 0]
 [0 0 5 0 0 0]
 [2 3 6 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[3 5 1 4 0 2]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 5 0 0 0]
 [0 0 0 0 0 0]
 [1 0 4 0 0 0]
 [2 3 6 0 0 0]]
[[4 5 2 0 0 0]
 [0 0 5 0 0 0]
 [2 3 6 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
